In [113]:
import os
import pandas as pd
import json
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
import nltk
import re

from collections import defaultdict

import numpy as np


from langdetect import detect
from langdetect import DetectorFactory
import pickle

import spacy

In [ ]:
big = pd.read('big.csv')

In [ ]:
#Create lemmatized body text column in data
stop_words = set(stopwords.words('english'))
custom_stop_words = ['fig','virus','il-','ifn','virus','cell','cells','data','model','medrxiv','data','disease','licence',
                    'certified','version','author','nc','nd','cc','rsv','reuse','peer','review','international','by',
                    'preprint','respondent','http','https','copyright','patient','introduction']

def stop_word_add(custom,words:set):
    for word in custom:
        words.add(word)
    return words

lemmatizer = WordNetLemmatizer()
def tokenize_and_lemmatize(text):
    doc = nlp(text)
    tokens = [word for word in doc.ents]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', str(token)) and len(str(token)) > 1 and str(token) not in stop_words:
            filtered_tokens.append(str(token))       
    stems = [lemmatizer.lemmatize(t) for t in filtered_tokens]
    l = ', '.join(stems)
    return l



In [ ]:
stop_words = stop_word_add(custom_stop_words,stop_words)

nlp = spacy.load('en_core_sci_lg')
nlp.max_length = 5000000

big['body_text_new'] = big.apply(lambda row: tokenize_and_lemmatize(row['body_text'].lower()), axis=1)
big['abstract_new'] = big.apply(lambda row: tokenize_and_lemmatize(row['abstract'].lower()), axis=1)


big['title'] = big['title'].str.lower()

#Export to csv file
print('saving to csv')
big.to_csv('data.csv',index=False)

In [34]:
# USEmodel = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# train = USEmodel(data.body_text)
# train_m = tf.train.Checkpoint(v=tf.Variable(train))

# train_m.f = tf.function( lambda  x: exported_m.v * x, input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])

# model = train_m.v.numpy()

# pickle.dump(model, open('full_tfidf_model.pk', 'wb'))

In [35]:
# count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
# count_data = count_vectorizer.fit_transform(data['body_text_new'])

In [36]:
# def SearchDocument(query,size=100):
#     q =[query]
#     Q_Train = USEmodel(q)
    
#     linear_similarities = linear_kernel(Q_Train, model).flatten() 
    
#     Top_index_doc = linear_similarities.argsort()[:-(size+1):-1]
#     #print(Top_index_doc)
#     linear_similarities.sort()
#     find = pd.DataFrame()
#     for i,index in enumerate(Top_index_doc):
#         find.loc[i,'index'] = str(index)
#         find.loc[i,'Paper_ID'] = data['paper_id'][index] 
#         find.loc[i,'Title'] = data['title'][index] 
#         find.loc[i,'abstract'] = data['abstract'][index]
#     for j,simScore in enumerate(linear_similarities[:-(size+1):-1]):
#         find.loc[j,'Score'] = simScore
        
#     if size==1:
#         if find.isnull().values.any():
#             print("Query: ",query,".  Title of the Research Paper is missing, Paper ID is:",find.loc[0,'Paper_ID'],"\n")
#         else:
#             print("Query: ",query,".  Title of the Research Paper:",find.loc[0,'Title'],"\n")
#     else:
#         return find